Скопировала функции по поиску вакансий в HH и SuperJob из второго домашнего задания

In [12]:
def search_hh():
    from bs4 import BeautifulSoup as bs
    import requests
    from pprint import pprint
    from user_agent import generate_user_agent

    main_link = 'https://hh.ru'
    title = input('Введите название вакансии: ')
    page_num = 0
    page_link = f'{main_link}/search/vacancy?clusters=true&area=1&search_field=name&enable_snippets=true&salary=&st=searchVacancy&text={title}&page={page_num}'
    response = requests.get(page_link,headers = {"User-Agent":generate_user_agent()})
    soup = bs(response.text,'html.parser')
    vacancies = []

    while soup.find('a', attrs = {'data-qa': 'pager-next'}) and page_num < 3:
        vacancy_descr = soup.find_all('div', attrs = {'class': 'vacancy-serp-item__row_header'})
        for vacancy in vacancy_descr:
            vacancy_data={}
            salary_splited = {}
    
            vacancy_link = vacancy.find('a', class_='bloko-link HH-LinkModifier').get('href')
            vacancy_name = vacancy.find('a').getText()
            vacancy_salary = vacancy.find('div',class_='vacancy-serp-item__sidebar').getText()
    
            vacancy_data['name'] = vacancy_name
            vacancy_data['link'] = vacancy_link
            vacancy_salary = vacancy_salary.replace('\xa0','')
            vacancy_salary = vacancy_salary.split(' ')
            if vacancy_salary[0] == 'от':
                salary_splited['min'] = int(vacancy_salary[1])
                salary_splited['max'] = 'none'
            if vacancy_salary[0] == 'до':                
                salary_splited['min'] = 'none'
                salary_splited['max'] = int(vacancy_salary[1])
            if '-' in vacancy_salary[0]:
                vacancy_salary[0] = vacancy_salary[0].split('-')
                salary_splited['min'] = int(vacancy_salary[0][0])
                salary_splited['max'] = int(vacancy_salary[0][1])
            salary_splited['conditions'] = vacancy_salary[-1]
            vacancy_data['salary'] = salary_splited    
            vacancies.append(vacancy_data)
        page_num += 1
        page_link = f'{main_link}/search/vacancy?clusters=true&area=1&search_field=name&enable_snippets=true&salary=&st=searchVacancy&text={title}&page={page_num}'
        response = requests.get(page_link,headers = {"User-Agent":generate_user_agent()})
        soup = bs(response.text,'html.parser')
    return vacancies


In [3]:
def search_sj():
    from bs4 import BeautifulSoup as bs
    import requests
    from pprint import pprint   
    from user_agent import generate_user_agent
    
    main_link = 'https://superjob.ru'
    title = input('Введите название вакансии: ')
    page_num = 1
    page_link = f'{main_link}/vacancy/search/?keywords={title}&geo%5Bt%5D%5B0%5D=4&page={page_num}'
    response = requests.get(page_link,headers = {"User-Agent":generate_user_agent()})
    soup = bs(response.text,'html.parser')
    vacancies = []

    while soup.find('a', attrs = {'class': 'f-test-link-Dalshe'}):
        vacancy_descr = soup.find_all('div', attrs = {'class': 'jNMYr'})
        for vacancy in vacancy_descr:
            vacancy_data={}
            salary_splited = {}
    
            vacancy_link = vacancy.find('a', class_='icMQ_').get('href')
            vacancy_name = vacancy.find('a').getText()
            vacancy_salary = vacancy.find('span',class_='_1OuF_').getText()
    
            vacancy_salary = vacancy_salary.split('\xa0')
        
            if vacancy_salary[0] == 'По договорённости':
                salary_splited['min'] = 'none'
                salary_splited['max'] = 'none'
                salary_splited['conditions'] = 'По договоренности' 
            elif vacancy_salary[0] == 'от':
                salary_splited['min'] = int(vacancy_salary[1] + vacancy_salary[2])
                salary_splited['max'] = 'none'
                salary_splited['conditions'] = vacancy_salary[3]
            elif vacancy_salary[0] == 'до':
                salary_splited['min'] = 'none'
                salary_splited['max'] = int(vacancy_salary[1] + vacancy_salary[2])
                salary_splited['conditions'] = vacancy_salary[3]
            elif len(vacancy_salary) == 3:
                salary_splited['min'] = int(vacancy_salary[0] + vacancy_salary[1])
                salary_splited['max'] = int(vacancy_salary[0] + vacancy_salary[1])
                salary_splited['conditions'] = vacancy_salary[2]
            elif len(vacancy_salary) > 3 and vacancy_salary[2] == '—':
                salary_splited['min'] = int(vacancy_salary[0] + vacancy_salary[1])
                salary_splited['max'] = int(vacancy_salary[3] + vacancy_salary[4])
                salary_splited['conditions'] = vacancy_salary[5]
            else:
                salary_splited['min'] = vacancy_salary
                   
            vacancy_data['name'] = vacancy_name
            vacancy_data['link'] = main_link + vacancy_link
            vacancy_data['salary'] = salary_splited   
            vacancies.append(vacancy_data)
    
        page_num += 1
        page_link = f'{main_link}/vacancy/search/?keywords={title}&geo%5Bt%5D%5B0%5D=4&page={page_num}'
        response = requests.get(page_link,headers = {"User-Agent":generate_user_agent()})
        soup = bs(response.text,'html.parser')
    
    return vacancies

In [3]:
#!pip install pymongo

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [26]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient('127.0.0.1',27017)
db = client['vacancies_db']

vacancies_db = db.vacancies_db

In [27]:
vacancies = search_hh()
n=0
for one_vacancy in vacancies:
    vacancies_db.update_one({'link': one_vacancy['link']}, {'$set': one_vacancy}, upsert = True)


Введите название вакансии: программист


In [31]:
for v in vacancies_db.find({}):
    pprint(v)

{'_id': ObjectId('5f5a7a3372ea589e8bf61bc2'),
 'age': 78,
 'author': 'Peter',
 'date': '14.06.1983',
 'tags': ['cool', 'hot', 'ice'],
 'text': 'is cool! Wildberry'}
{'_id': ObjectId('5f5a7c1772ea589e8bf61bc4'),
 'link': 'https://hh.ru/vacancy/39138787?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'Frontend-разработчик',
 'salary': {'conditions': 'руб.', 'max': 140000, 'min': 100000}}
{'_id': ObjectId('5f5a7c1772ea589e8bf61bc5'),
 'link': 'https://hh.ru/vacancy/37674249?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'Программист-разработчик (python)',
 'salary': {'валюта': 'руб.', 'максимум': '180000', 'минимум': '100000'}}
{'_id': ObjectId('5f5a7c1772ea589e8bf61bc6'),
 'link': 'https://hh.ru/vacancy/39105275?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'PHP-разработчик',
 'salary': {'conditions': 'руб.', 'max': 'none', 'min': 150000}}
{'_id': ObjectId('5f5a7c1772ea589e8bf61

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

In [34]:
def find_vacancies(wanted_salary):
    for v in vacancies_db.find({'$or': [{'salary.min': { '$gt':wanted_salary}}, {'salary.max': { '$gt':wanted_salary}}]}):
        pprint(v)

In [35]:
find_vacancies(100000)

{'_id': ObjectId('5f5a7c1772ea589e8bf61bc4'),
 'link': 'https://hh.ru/vacancy/39138787?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'Frontend-разработчик',
 'salary': {'conditions': 'руб.', 'max': 140000, 'min': 100000}}
{'_id': ObjectId('5f5a7c1772ea589e8bf61bc6'),
 'link': 'https://hh.ru/vacancy/39105275?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'PHP-разработчик',
 'salary': {'conditions': 'руб.', 'max': 'none', 'min': 150000}}
{'_id': ObjectId('5f5a7c1772ea589e8bf61bc7'),
 'link': 'https://hh.ru/vacancy/38466734?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'Программист 1С',
 'salary': {'conditions': 'руб.', 'max': 'none', 'min': 150000}}
{'_id': ObjectId('5f5a7c1772ea589e8bf61bc8'),
 'link': 'https://hh.ru/vacancy/38243034?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'name': 'Программист 1С (БП 3 и ЗУП 3)',
 'salary': {'conditions': '